# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fde5ca12310>
├── 'a' --> tensor([[-0.3188,  1.0491, -0.1608],
│                   [ 0.2064, -0.1857,  1.4721]])
└── 'x' --> <FastTreeValue 0x7fde5ca122e0>
    └── 'c' --> tensor([[ 0.2583, -0.8978,  0.7120, -0.7495],
                        [-0.1637, -1.0891, -0.2724, -0.2472],
                        [ 0.1709,  0.8224,  0.3530, -2.2221]])

In [4]:
t.a

tensor([[-0.3188,  1.0491, -0.1608],
        [ 0.2064, -0.1857,  1.4721]])

In [5]:
%timeit t.a

74.9 ns ± 1.31 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fde5ca12310>
├── 'a' --> tensor([[ 0.9091, -0.4999,  1.3576],
│                   [-0.6347, -1.0243,  1.3367]])
└── 'x' --> <FastTreeValue 0x7fde5ca122e0>
    └── 'c' --> tensor([[ 0.2583, -0.8978,  0.7120, -0.7495],
                        [-0.1637, -1.0891, -0.2724, -0.2472],
                        [ 0.1709,  0.8224,  0.3530, -2.2221]])

In [7]:
%timeit t.a = new_value

82.5 ns ± 0.537 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.3188,  1.0491, -0.1608],
               [ 0.2064, -0.1857,  1.4721]]),
    x: Batch(
           c: tensor([[ 0.2583, -0.8978,  0.7120, -0.7495],
                      [-0.1637, -1.0891, -0.2724, -0.2472],
                      [ 0.1709,  0.8224,  0.3530, -2.2221]]),
       ),
)

In [10]:
b.a

tensor([[-0.3188,  1.0491, -0.1608],
        [ 0.2064, -0.1857,  1.4721]])

In [11]:
%timeit b.a

69.3 ns ± 1.25 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.4341, -0.8132,  0.0516],
               [ 1.9051, -0.3286, -1.6303]]),
    x: Batch(
           c: tensor([[ 0.2583, -0.8978,  0.7120, -0.7495],
                      [-0.1637, -1.0891, -0.2724, -0.2472],
                      [ 0.1709,  0.8224,  0.3530, -2.2221]]),
       ),
)

In [13]:
%timeit b.a = new_value

565 ns ± 5.5 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

972 ns ± 16 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.7 µs ± 47.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

162 µs ± 1.77 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

162 µs ± 562 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fde26bcda30>
├── 'a' --> tensor([[[-0.3188,  1.0491, -0.1608],
│                    [ 0.2064, -0.1857,  1.4721]],
│           
│                   [[-0.3188,  1.0491, -0.1608],
│                    [ 0.2064, -0.1857,  1.4721]],
│           
│                   [[-0.3188,  1.0491, -0.1608],
│                    [ 0.2064, -0.1857,  1.4721]],
│           
│                   [[-0.3188,  1.0491, -0.1608],
│                    [ 0.2064, -0.1857,  1.4721]],
│           
│                   [[-0.3188,  1.0491, -0.1608],
│                    [ 0.2064, -0.1857,  1.4721]],
│           
│                   [[-0.3188,  1.0491, -0.1608],
│                    [ 0.2064, -0.1857,  1.4721]],
│           
│                   [[-0.3188,  1.0491, -0.1608],
│                    [ 0.2064, -0.1857,  1.4721]],
│           
│                   [[-0.3188,  1.0491, -0.1608],
│                    [ 0.2064, -0.1857,  1.4721]]])
└── 'x' --> <FastTreeValue 0x7fddb26e6f40>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

37.4 µs ± 217 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fddb26e7d30>
├── 'a' --> tensor([[-0.3188,  1.0491, -0.1608],
│                   [ 0.2064, -0.1857,  1.4721],
│                   [-0.3188,  1.0491, -0.1608],
│                   [ 0.2064, -0.1857,  1.4721],
│                   [-0.3188,  1.0491, -0.1608],
│                   [ 0.2064, -0.1857,  1.4721],
│                   [-0.3188,  1.0491, -0.1608],
│                   [ 0.2064, -0.1857,  1.4721],
│                   [-0.3188,  1.0491, -0.1608],
│                   [ 0.2064, -0.1857,  1.4721],
│                   [-0.3188,  1.0491, -0.1608],
│                   [ 0.2064, -0.1857,  1.4721],
│                   [-0.3188,  1.0491, -0.1608],
│                   [ 0.2064, -0.1857,  1.4721],
│                   [-0.3188,  1.0491, -0.1608],
│                   [ 0.2064, -0.1857,  1.4721]])
└── 'x' --> <FastTreeValue 0x7fde5ca1f9a0>
    └── 'c' --> tensor([[ 0.2583, -0.8978,  0.7120, -0.7495],
                        [-0.1637, -1.0891, -0.2724, -0.2472],
                 

In [23]:
%timeit t_cat(trees)

35.3 µs ± 266 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

61.9 µs ± 272 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.3188,  1.0491, -0.1608],
                [ 0.2064, -0.1857,  1.4721]],
       
               [[-0.3188,  1.0491, -0.1608],
                [ 0.2064, -0.1857,  1.4721]],
       
               [[-0.3188,  1.0491, -0.1608],
                [ 0.2064, -0.1857,  1.4721]],
       
               [[-0.3188,  1.0491, -0.1608],
                [ 0.2064, -0.1857,  1.4721]],
       
               [[-0.3188,  1.0491, -0.1608],
                [ 0.2064, -0.1857,  1.4721]],
       
               [[-0.3188,  1.0491, -0.1608],
                [ 0.2064, -0.1857,  1.4721]],
       
               [[-0.3188,  1.0491, -0.1608],
                [ 0.2064, -0.1857,  1.4721]],
       
               [[-0.3188,  1.0491, -0.1608],
                [ 0.2064, -0.1857,  1.4721]]]),
    x: Batch(
           c: tensor([[[ 0.2583, -0.8978,  0.7120, -0.7495],
                       [-0.1637, -1.0891, -0.2724, -0.2472],
                       [ 0.1709,  0.8224,  0.3530, -2.2221]],
         

In [26]:
%timeit Batch.stack(batches)

90 µs ± 496 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.3188,  1.0491, -0.1608],
               [ 0.2064, -0.1857,  1.4721],
               [-0.3188,  1.0491, -0.1608],
               [ 0.2064, -0.1857,  1.4721],
               [-0.3188,  1.0491, -0.1608],
               [ 0.2064, -0.1857,  1.4721],
               [-0.3188,  1.0491, -0.1608],
               [ 0.2064, -0.1857,  1.4721],
               [-0.3188,  1.0491, -0.1608],
               [ 0.2064, -0.1857,  1.4721],
               [-0.3188,  1.0491, -0.1608],
               [ 0.2064, -0.1857,  1.4721],
               [-0.3188,  1.0491, -0.1608],
               [ 0.2064, -0.1857,  1.4721],
               [-0.3188,  1.0491, -0.1608],
               [ 0.2064, -0.1857,  1.4721]]),
    x: Batch(
           c: tensor([[ 0.2583, -0.8978,  0.7120, -0.7495],
                      [-0.1637, -1.0891, -0.2724, -0.2472],
                      [ 0.1709,  0.8224,  0.3530, -2.2221],
                      [ 0.2583, -0.8978,  0.7120, -0.7495],
                      [-0.1637, -

In [28]:
%timeit Batch.cat(batches)

162 µs ± 883 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

366 µs ± 4.22 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
